## **Lesk++ algorithm**
Dieke Oele, Gertjan van Noord

Published in GWC 2018

Computer Science

paper credit: [Simple Embedding-based word sense disambiguation](https://www.semanticscholar.org/paper/Simple-Embedding-Based-Word-Sense-Disambiguation-Oele-Noord/25f93681a329787831d3fcf9d268f8820b0deaf6).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
'''Initializaing data structure MSW'''
!gdown --id 1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a 

Downloading...
From: https://drive.google.com/uc?id=1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a
To: /content/MSW.py
100% 642/642 [00:00<00:00, 1.15MB/s]


In [3]:
# 要叫出leveled_list 
# 要叫出w2v的訓練文檔 
# w2v沒有UNK，所以看要不要fall back to character embedding
import pickle, json 
W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
W2VSEGDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_SegTexts'
List_file = f'{W2VDIR}/MSW_leveled_list3.pkl'
with open (List_file, 'rb') as F:
  leveled_list = pickle.load(F)

In [4]:
Embeddings = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources/TrainedC_embeddings.wv'
from gensim.models import Word2Vec
w2vmodel = Word2Vec.load(Embeddings)

###  Gloss_dict structure (for reference)
斷詞；去除punctuations 
```
f'{W2VDIR}/Gloss_dict.json'
```

```
├── Gloss_dict
│   ├── [{word}]
│   │   ├── [{sense}]
│   │   │    ├── ["gloss"] # segmented 
│   │   │    ├── ["Gemb"]  # avg of seg gloss embedding 
│   │   │    ├── ["category"] 
│   │   │    ├── ["senseidx"] 
```


In [ ]:
# Lmodel.GlossDict['大氣層']

In [ ]:
# similarity # dimension差很多會怎樣嗎？

傻眼我把sensename當Gloss Dict的key

結果剛剛才發現sensename會有重複的
### Sol1:
#### 如果有撞sensename名的，那我就讓他們全部算在同一個名字底下
#### 考量：通常撞名的是main，而main的意義通常會跟category底下某個意義重疊，而他們又會respectively屬於main, {category}兩類，因此計算acc時會降低。但當然這有點取巧...

### Sol2:
#### 如果有撞sensename名而且GlossDict裡找不到的那就不要測 QQ


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
class LeskPP:
  W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
  W2VSEGDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_SegTexts'
  EmbPath= f'{W2VDIR}/TrainedC_embeddings.wv'
  # LeveledPath= f'{self.W2VDIR}/MSW_leveled_list3.pkl'
  GlossPath= f'{W2VDIR}/Gloss_dict.json'
  DefaultPath = f'{W2VDIR}/Lesked_GlossD.pkl'
  ##### fixing bugs... ##### 
  IDSPath = f'{W2VDIR}/ID_to_sensename.pkl'
  CatPath = f'{W2VDIR}/CategoryDicts.pkl'
  EMBDIM = 200 # w2v 時的訓練維度
  def __init__(self):
      pass
  def load(self):
      # load w2v dict
      print('[Info] Notice: please load MSW class in advance')
      print('[Info] Loading...')
      from gensim.models import Word2Vec
      import pickle, json
      # load embeddings
      w2vmodel = Word2Vec.load(self.EmbPath)
      self.w2v = w2vmodel.wv # 可以只存.wv
      with open (self.GlossPath, 'rb') as F:
          self.GlossDict = json.load(F)
      with open (self.IDSPath, 'rb') as fp:
          self.ID_to_sensename = pickle.load(fp)
      with open (self.CatPath, 'rb') as f:
          self.categoryDicts = pickle.load(f) 
      print('[Info] Finished loading.')
  def makeGsj(self, target, target_sense):
      # average gloss embeddings 
      # gloss[f'{word}'][f'{sensename}']['gloss'] =  seg_gloss
      # caching 
      Gemb = self.GlossDict[target][target_sense]['Gemb']#
      if isinstance(Gemb, np.ndarray):
          return Gemb
      glossvecs = []
      Gloss = self.GlossDict[target][target_sense]['gloss']
      for token in Gloss:
          try: 
              glossvecs.append(self.w2v[token])
          except: pass
      if len(glossvecs) == 0:
        return np.zeros(self.EMBDIM)
      # print(f'[Info] GlossLen: {len(glossvecs)}')
      vecs = np.vstack(glossvecs)
      avgGsj = np.mean(vecs, axis=0)
      # caching
      self.GlossDict[target][target_sense]['Gemb'] = avgGsj
      return avgGsj

  def makeCw(self, testsent, limit = 2):
      # average context embeddings (context in the input sentence)
      ctxvecs = []
      for token in testsent:
        try:
          ctxvecs.append(self.w2v[token])
          # print(token)
        except: pass
      if len(ctxvecs) < limit:
        # print(f'No enough context words in the input sentence to make Context Vectors.')
        return np.zeros(self.EMBDIM)   
      vecs = np.vstack(ctxvecs)
      avgCw = np.mean(vecs, axis=0)
      return avgCw
  def getLsj(self, target_sense):
      # get one sense embedding 
      # from w2v dictionary 
      if target_sense in self.w2v: 
        return self.w2v[target_sense]
      return np.zeros(self.EMBDIM)   
  def compute_score(self, Cw, Gsj, Lsj):
      # score(w,sj) = cos(Gsj, Cw) + cos(Lsj, Cw)
      Cw = Cw.reshape(1, -1) 
      Gsj = Gsj.reshape(1, -1)      
      Lsj = Lsj.reshape(1, -1) 
      scorearr = cosine_similarity(Cw, Gsj) + cosine_similarity(Lsj, Cw)
      return scorearr[0][0]
  def load_categoryD(self, target: str):
    # 存錯形式了QQ 
    return self.categoryDicts[target]
    print('* Finished loading Category Dictinoary.')
  def test_batch(self, 
                 testsents: list,
                 target: str,
                 verbose: int):
    # save info 
    y_trues = []
    y_hats = []
    # test in batch and compute accuracy 
    # testsents: a list of ([list of tokens], gold_label)
    print('[Info] Start testing...')
    print(f'* ----- Target word: {target} -----')
    Accurates = 0 
    TESTSIZE = 0
    catD = self.load_categoryD(target = target)
    ## 如果只有一個category就返回不計入
    if len(set(catD.values())) <=1:
      print(f"{target} has <=1 categories. Categorical accuaracy can't be computed.")
      return 
    not_contain = 0 
    for testsentpair in testsents:
        testsent, gold_label = testsentpair 
        ##### !!! fixing bugs !!! ##### 
        if gold_label not in catD:
          '''Sol2'''
          continue
          '''
          # Sol1: 找到gold_label原本對應的sense
          # 以sense去self.GlossDict[f'{target}'][f'{sensename}']找idx然後換掉gold_label 
          print(self.ID_to_sensename[target])
          orig_sense = self.ID_to_sensename[target][gold_label]
          for idkey in self.ID_to_sensename[target].keys():
            if self.ID_to_sensename[target][idkey] == orig_sense\
              and idkey in self.catD:
              print(f'[Info] {target}: {gold_label} replaced by {idkey} with same sensename {orig_sense}.')
              gold_label = idkey
          '''
        ##############################
        scoredict = self.test(testsent, target)
        if scoredict == -1:
            not_contain += 1
            continue
        TESTSIZE += 1
        prediction = scoredict[0]
        ## prediction  
        prediction_ = prediction[0].split('_')
        yhat_sense = prediction_[0] # 蘋果園區
        yhat_label = int(prediction_[1]) # 3
        ## Lesk++只算大category的正確率
        if verbose:
            print('* Testsent:', ' '.join(testsent)) 
            print('* Gold category: ', catD[gold_label], '* Pred category:', catD[yhat_label])
        # print 前三名candidates
            print(*scoredict[:3], sep = "\t\n")
        
        y_hats.append(catD[yhat_label])
        y_trues.append(catD[gold_label])
        if catD[yhat_label] == catD[gold_label]:
            Accurates += 1

    if TESTSIZE == 0: 
        print('[Warning] TESTSIZE = 0')
        return 
    Accuracy = round(Accurates/TESTSIZE, 4) 
    print(f'[Info] Total {TESTSIZE} valid sentences')
    print(f'[Info] *** Category Accuracy: {Accuracy} ***')    
    print('---------------------')  
     
    return Accurates, TESTSIZE, y_trues, y_hats 
  def test(self, 
           testsent: list, 
           target: str):
      '''
      testsent: a list of tokens (pre-segmented tokens)
      '''
      # mode = 'simple', 'complex'
      Cw = self.makeCw(testsent)
      if target not in testsent:
          # print(f'[Warning] test sent does not contain target {target}.')
          return -1 
      if np.all((Cw == 0)): # not None
          # meaning that <= 2: 
          print(f'[Warning] No enough context words in the input sentence to make Context Vectors.')
          return -1
      senselist = self.GlossDict[target].keys()
      sense_scores = {}
      available_senses = []
      available_sense = 0 
      ScoreDict = {}
      for sense in senselist:
          score = -1  # indicating no essential embeddings available among Lsj, Gsj, Cw
          sensedict = self.GlossDict[target][sense]
          id = sensedict['senseidx']   
          Gs_name = sense
          Gsj = self.makeGsj(target = target, 
                             target_sense = Gs_name) # Gloss dict是用sensename當keys
          Ls_name =f'{target}_{id}'    # 
          Lsj = self.getLsj(Ls_name)  # EMBDIM= 200
          # print(Gs_name, Gsj)
          # print(Ls_name, Lsj)
          if  np.all((Lsj != 0)) and np.all((Gsj != 0)): 
              available_sense += 1 
              score = self.compute_score(Cw = Cw, 
                                     Lsj = Lsj,
                                     Gsj = Gsj)    
          # print(f'{sense} Score:', score)
          ScoreDict[f'{sense}_{id}'] = score
      # print(f'# Recorded Senses: {len(senselist)},# Available Senses: {available_sense}')
      return sorted(ScoreDict.items(), key=lambda x:-x[1]) # sort by score, reverse
  def saveGD(self, path= DefaultPath):
    with open(path, 'rb') as f:
         pickle.dump(self.GlossDict, f)
    print('[Info] File Saved to pickle.')

In [6]:
Lmodel = LeskPP()

In [7]:
Lmodel.load()

[Info] Notice: please load MSW class in advance
[Info] Loading...
[Info] Finished loading.


In [ ]:
print(*Lmodel.GlossDict['蘋果'])

蘋果 蘋果公司 蘋果園區 蘋果唱片公司 蘋果日報 蘋果日報 (香港) 蘋果日報 (臺灣) 蘋果日報慈善基金 蘋果 (電影)


In [ ]:
# w2v = Word2Vec.load(Embeddings).wv

In [8]:
import matplotlib as mpl
import matplotlib.font_manager as fm
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
fm.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

--2022-01-12 19:00:44--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 142.251.107.101, 142.251.107.100, 142.251.107.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.107.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osr3via2nju8eb2vrphfejrq9jvbuern/1642014000000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2022-01-12 19:00:49--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osr3via2nju8eb2vrphfejrq9jvbuern/1642014000000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 173.194.215.132, 2607:f8b0:400c:c0c::84
Connecting to doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)|1

### Test corpus loading

In [9]:
with open(f'{W2VDIR}/Test_Corpus.pkl', 'rb') as f:
  Test_Corpus = pickle.load(f)

In [10]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline
# /content/drive/MyDrive/110-1ComSem_final/tfidf_resources/Pred_ConMatrices
CONFDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources/ConfMatrices'

In [ ]:
target = '媽祖'
Lmodel.test_batch(testsents = Test_Corpus[target], 
            target = target, verbose = 1)

In [24]:
import pandas as pd
W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
DF = pd.read_csv(f'{W2VDIR}/msw_eda.csv')
Accurates = {'3 ~ 5':0, '< 3':0, '6 ~ 15':0}
Totals = {'3 ~ 5':0, '< 3':0, '6 ~ 15':0}
for i in range(len(DF)):
  mswword = DF.loc[i]['target']
  Res = Lmodel.test_batch(testsents = Test_Corpus[mswword], 
                          target = mswword, verbose = 0)
  if Res is not None and DF.loc[i]['csense#'] > 1:
    accurates, testsize, y_trues, y_hats = Res
    Type = DF.loc[i]['csense_cat']
    Accurates[Type] += accurates
    Totals[Type] += testsize

[Info] Total 122 valid sentences
[Info] *** Category Accuracy: 0.7131 ***
---------------------
[Info] Start testing...
* ----- Target word: 地獄 -----
[Info] Total 24 valid sentences
[Info] *** Category Accuracy: 0.4583 ***
---------------------
[Info] Start testing...
* ----- Target word: 電信 -----
[Info] Total 30 valid sentences
[Info] *** Category Accuracy: 0.8667 ***
---------------------
[Info] Start testing...
* ----- Target word: 鼎 -----
[Info] Total 47 valid sentences
[Info] *** Category Accuracy: 0.6383 ***
---------------------
[Info] Start testing...
* ----- Target word: 東海 -----
[Info] Total 67 valid sentences
[Info] *** Category Accuracy: 0.8507 ***
---------------------
[Info] Start testing...
* ----- Target word: 東京 -----
[Info] Total 160 valid sentences
[Info] *** Category Accuracy: 0.7812 ***
---------------------
[Info] Start testing...
* ----- Target word: 豆蔻 -----
[Info] Total 8 valid sentences
[Info] *** Category Accuracy: 0.75 ***
---------------------
[Info] Start 

In [25]:
print(Accurates)
Totals

{'3 ~ 5': 3549, '< 3': 453, '6 ~ 15': 1087}


{'3 ~ 5': 5620, '6 ~ 15': 2243, '< 3': 713}

In [27]:
keys = Totals.keys()
type_accuracy = {key:Accurates[key]/Totals[key] for key in keys}
type_accuracy
# Lesk ++ 
# 8576
# 5089
# Final word count: 154
# Total overall accuracy: 0.5934

{'3 ~ 5': 0.6314946619217082,
 '6 ~ 15': 0.4846188140882746,
 '< 3': 0.635343618513324}

In [ ]:
wordlist = list(Test_Corpus.keys())
TOT_Testsize = 0
TOT_Accurates = 0
LeskAccD = {}

for msw in wordlist:
  Res = Lmodel.test_batch(testsents = Test_Corpus[msw], target = msw, verbose = 0)
  if Res is not None:
    print(Res)
    print('-------------------------')
    accurates, testsize, y_trues, y_hats = Res
    #LeskAccD['蘋果'] = ()
    LeskAccD[msw] = (testsize, round(accurates/testsize, 4))
    TOT_Testsize += testsize
    TOT_Accurates += accurates
    CAcc = round(accurates/testsize, 4)

    ############################
    categoryD = list(set(y_trues))
    Cconf = confusion_matrix(y_trues, y_hats, labels = categoryD)
    ## save confusion matrix visualization ###
      
    disp1 = ConfusionMatrixDisplay(Cconf, 
                                    display_labels = categoryD)
    disp1.plot(xticks_rotation = 40)
    plt.gcf().subplots_adjust(bottom=0.5)
    plt.title(f'Coarse-grained CM: {msw} | {CAcc:.4f}')
    plt.savefig(f'{CONFDIR}/{msw}_CCM.png', dpi=150, bbox_inches='tight')

# print(*LeskAccD, sep = '\n')
print(TOT_Testsize)
print(TOT_Accurates)
print(f'Final word count: {len(LeskAccD)}')
print(f'Total accuracy: {TOT_Accurates/TOT_Testsize:.4f} ')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with open (f'{W2VDIR}/LeskAccRecords.json', 'w') as f:
      json.dump(LeskAccD, f) 

In [ ]:
w2v = Lmodel.w2v
similar_words = {
    search_term:
    [item[0] for item in w2v.most_similar([search_term], topn=8)]
    for search_term in
    ['蘋果_1', '蘋果_2', '蘋果_3', '蘋果_4', '蘋果_5']
}
for x in similar_words:
  print(x, similar_words[x])

蘋果_1 ['品種', '葡萄_1', '番茄_2', '水果', '樹', '馴化', '楊梅_1', '新鮮']
蘋果_2 ['微軟', 'Macintosh', '喬布斯', 'Lisa', 'Mac', 'Micro', 'iMac', 'Prose']
蘋果_3 ['蘋果_4', '園區', 'McCarthyBuildingCompanies', '田溪', '麥卡錫', '惠普', '書版', 'Premier']
蘋果_4 ['蘋果_3', '園區', 'McCarthyBuildingCompanies', '田溪', '域迅', '書版', '港鐵', '希代']
蘋果_5 ['爲盆', 'SkyOne', '生死之交', '邯鄲市', '移除掉', '看懂', 'Gabapentin', '善行']


In [ ]:
'''
from sklearn.decomposition import PCA
print(X)
# 2. PCA 
pca = PCA(n_components=2)
X_res = pca.fit_transform(X)
my_colors = {0:'indigo', 1:'coral',2:'limegreen',3:'deepskyblue'}
for i,(j, text) in enumerate(zip(X_res, words)):
    # look for the color based on vocabulary, if not found in vocubulary, then black is returned.
    plt.scatter(X_res[i,0] , X_res[i,1], color = my_colors.get(cluster_idxes[i], 'dimgray'))
    plt.annotate(text, (X_res[i,0] , X_res[i,1]), fontsize = 14)
plt.title('Embedding Visualization: Kmeans')
plt.rcParams["figure.figsize"] = (12, 10)
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.grid(True)
plt.show() 
'''

In [ ]:
target = '蘋果'
# testsent, label = Test_Corpus[target][10]
# onetest = Lmodel.test(testsent = testsent, target = target)
Lmodel.test_batch(testsents = Test_Corpus[target], 
            target = target, verbose = 1)

[Info] Start testing...
* ----- Target word: 蘋果 -----
* Finished making Category Dictinoary.
* Testsent: 人們 根據 需求 的 不同 口感 用途 比如 烹飪 生吃 釀 蘋果 酒 等 培育 不同 的 品種 已 知 有 超過 7 , 500 個 蘋果 品種 擁有 一 系列 人們 需要 的 不同 特性
* Gold category:  main * Pred category: main
('蘋果_1', 0.86337566)	
('蘋果園區_3', 0.31219465)	
('蘋果唱片公司_4', 0.21983379)
* Testsent: 蘋果 樹 及 其 果實 很 容易 出現 許多 真菌 細菌 和 害蟲 問題 這些 問題 可以 通過 多 種 有機 和 非 有機 手段 加以 控制
* Gold category:  main * Pred category: main
('蘋果_1', 0.62442535)	
('蘋果園區_3', 0.011808308)	
('蘋果唱片公司_4', -0.060458336)
* Testsent: 中國 土生 蘋果 屬 植物 在 古代 又 稱 柰 或 林檎
* Gold category:  main * Pred category: main
('蘋果_1', 1.0520942)	
('蘋果園區_3', 0.044983983)	
('蘋果日報_5', 0.040838353)
* Testsent: 一些 地方 的 消費者 像 日本 就 更 喜歡 較 大 的 蘋果 21 ⁄ 4 英寸 57 cm 以下 的 蘋果 通常 用於 製作 果汁 並 不 常 拿來 賣
* Gold category:  main * Pred category: main
('蘋果_1', 0.6625161)	
('蘋果唱片公司_4', 0.20034541)	
('蘋果園區_3', 0.19724044)
* Testsent: 基因 組 2010年 意大利 一 家 財團 宣佈 他們 與 華盛頓 州立 大學 的 園藝 基因組 學家 合作 對 金冠 蘋果 的 完整 基因組 進行 了 測序
* Gold category:  main *

(130, 147)

In [ ]:
target = '荔枝'
# testsent, label = Test_Corpus[target][10]
# onetest = Lmodel.test(testsent = testsent, target = target)
Lmodel.test_batch(testsents = Test_Corpus[target], 
            target = target, verbose = 1)

In [ ]:
target = '荔枝'
Lmodel.categoryDicts[target]
# Lmodel.test_batch(testsents = Test_Corpus[target], 
# target = target, verbose = 1)

{1: 'main',
 2: '傳媒',
 3: '門戶',
 4: '門戶',
 5: '門戶',
 6: '門戶',
 7: '地理',
 8: '地理',
 9: '地理',
 10: '地理',
 11: '地理',
 12: '地理',
 13: '地理',
 14: '地理',
 15: '地理',
 16: '地理',
 17: '植物',
 18: '植物',
 19: '植物',
 20: '食物',
 21: '歌曲'}

In [ ]:
# Nice: 羅馬，蘋果，
# 非常糟: 獨立

In [ ]:
import random
target = '龍'
rand = random.randint(0, len(Test_Corpus[target]))
testsent, label = Test_Corpus[target][rand]
print(testsent)
print("**Gold label:", label)
Lmodel.test(testsent = testsent, 
            target = target)

In [ ]:
len(Test_Corpus['黃油'])

18

In [ ]:
Cw = Lmodel.makeCw(testsent)
Gsj = Lmodel.makeGsj('蘋果', '蘋果')
Lsj = Lmodel.getLsj('蘋果_1')
Cw = Cw.reshape(1, -1) 
Lsj = Lsj.reshape(1, -1) 
Gsj = Gsj.reshape(1, -1) 
# ctxvecs = []

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


array([[0.29010203]], dtype=float32)

##**Load test corpus for test**

In [ ]:
import re
for key in Test_Corpus.keys():
  sentences = Test_Corpus[key]
  Test_Corpus[key] = []
  for testpair in sentences:
    sent, label = testpair
    cleaned_sent = [re.sub(fr'{key}_\d+', key, token) for token in sent]
    testpair = (cleaned_sent, label)
    Test_Corpus[key].append(testpair)
with open(f'{W2VDIR}/Test_Corpus.pkl', 'wb') as f:
  pickle.dump(Test_Corpus, f)

In [ ]:
'''
# [工人智慧]
for i in range(len(Test_Corpus['荔枝'])):
  testsent = Test_Corpus['荔枝'][i][0]
  if Test_Corpus['荔枝'][i][-1] in (1,2,3,4,5):
    Test_Corpus['荔枝'][i] = (testsent,1) # 全部都是main的語意
with open(f'{W2VDIR}/Test_Corpus.pkl', 'wb') as f:
  pickle.dump(Test_Corpus, f)
'''